<a href="https://colab.research.google.com/github/mahault/Quantum-C/blob/main/Quantum_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.1/56.1 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 930.0/930.0 kB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 53.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 167.9/167.9 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 48.7 MB/s eta 0:00:00


In [ ]:
import pennylane as qml
from pennylane import numpy as np

# Quantum device setup
num_qubits = 4
dev = qml.device("default.qubit", wires=num_qubits)

# QCNN model
@qml.qnode(dev)
def qcnn(params, data):
    # Encoding: Rotate each qubit based on input data
    for i in range(num_qubits):
        qml.RY(data[i], wires=i)

    # Convolution layer
    for i in range(0, num_qubits, 2):  # Apply to pairs of qubits
        qml.CRX(params[0], wires=[i, i + 1])

    # Pooling layer: Reduce qubits by controlled measurement
    for i in range(0, num_qubits, 2):
        qml.CNOT(wires=[i, i + 1])
        qml.PauliZ(wires=i + 1)  # Example: some transformation

    # Fully connected layer
    qml.RY(params[1], wires=0)
    qml.RY(params[2], wires=2)

    # Measurement
    return [qml.expval(qml.PauliZ(i)) for i in range(2)]

def interpret_qcnn_output(output):
    """
    Convert QCNN expectation values into probabilities and classify.
    """
    probabilities = [(1 + val) / 2 for val in output]  # Convert to probabilities
    predicted_class = np.argmax(probabilities)  # Class with highest probability
    return probabilities, predicted_class

# Example input data
data = [0.1, 0.9, 0.4, 0.7]  # Encoded data
params = [0.5, 1.2, -0.7]  # Randomly initialized parameters

# Run QCNN
output = qcnn(params, data)
print("QCNN Output:", output)
probabilities, predicted_class = interpret_qcnn_output(output)
print("QCNN Output (Probabilities):", probabilities)
print("Predicted Class:", predicted_class)


QCNN Output: [0.2899258625260107, 0.6186945888769104]
QCNN Output (Probabilities): [0.6449629312630054, 0.8093472944384552]
Predicted Class: 1
